In [1]:
from lightgbm import LGBMRegressor
from hyperopt import fmin, tpe, hp
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np
import gc

<h3>NA imputed with 0</h3>

In [2]:
data = pd.read_pickle('./data/data.pkl')

X_train = data[data.date_block_num < 33].drop(['item_cnt_month', 'ID'], axis=1)
Y_train = data[data.date_block_num < 33]['item_cnt_month']
X_valid = data[data.date_block_num == 33].drop(['item_cnt_month', 'ID'], axis=1)
Y_valid = data[data.date_block_num == 33]['item_cnt_month']
X_test = data[data.date_block_num == 34].drop(['item_cnt_month', 'ID'], axis=1)

del data
gc.collect()

0

In [3]:
parameters = {
    'num_leaves': 2000, 'max_depth': 19, 'max_bin': 107, 'n_estimators': 3747,
    'bagging_freq': 1, 'bagging_fraction': 0.7135681370918421,
    'feature_fraction': 0.49446461478601994, 'min_data_in_leaf': 88, 
    'learning_rate': 0.015980721586917768, 'num_threads': 3, 
    'min_sum_hessian_in_leaf': 6,
    'verbosity' : 1,
    'boost_from_average' : 'true',
    'boost' : 'gbdt',
    'metric' : 'rmse'}

model = LGBMRegressor(**parameters)
    
model.fit(
    X_train,
    Y_train, 
    eval_metric="rmse", 
    eval_set=[(X_valid, Y_valid)], 
    verbose=False,
    early_stopping_rounds = 100)

LGBMRegressor(bagging_fraction=0.7135681370918421, bagging_freq=1, boost='gbdt',
              boost_from_average='true', boosting_type='gbdt',
              class_weight=None, colsample_bytree=1.0,
              feature_fraction=0.49446461478601994, importance_type='split',
              learning_rate=0.015980721586917768, max_bin=107, max_depth=19,
              metric='rmse', min_child_samples=20, min_child_weight=0.001,
              min_data_in_leaf=88, min_split_gain=0.0,
              min_sum_hessian_in_leaf=6, n_estimators=3747, n_jobs=-1,
              num_leaves=2000, num_threads=3, objective=None, random_state=None,
              reg_alpha=0.0, reg_lambda=0.0, silent=True, subsample=1.0,
              subsample_for_bin=200000, subsample_freq=0, ...)

In [4]:
data = pd.read_csv("./data/test.csv")
data['item_cnt_month'] = model.predict(X_test).clip(0, 20)
data[['ID', 'item_cnt_month']].to_csv("./features/feature_lgbm_tuned.csv", index = False)

In [5]:
pred = model.predict(pd.concat((X_train, X_valid))).clip(0, 20)
data = pd.concat((X_train, X_valid))
data['item_cnt_month'] = pred
data.to_csv("./features_train/feature_lgbm_tuned.csv")

<h3>NA not imputed</h3>

In [6]:
data = pd.read_pickle('./data/data_na.pkl')

X_train = data[data.date_block_num < 33].drop(['item_cnt_month', 'ID'], axis=1)
Y_train = data[data.date_block_num < 33]['item_cnt_month']
X_valid = data[data.date_block_num == 33].drop(['item_cnt_month', 'ID'], axis=1)
Y_valid = data[data.date_block_num == 33]['item_cnt_month']
X_test = data[data.date_block_num == 34].drop(['item_cnt_month', 'ID'], axis=1)

del data
gc.collect()

50

In [7]:
parameters = {
    'num_leaves': 2000, 'max_depth': 19, 'max_bin': 107, 'n_estimators': 3747,
    'bagging_freq': 1, 'bagging_fraction': 0.7135681370918421,
    'feature_fraction': 0.49446461478601994, 'min_data_in_leaf': 88, 
    'learning_rate': 0.015980721586917768, 'num_threads': 3, 
    'min_sum_hessian_in_leaf': 6,
    'verbosity' : 1,
    'boost_from_average' : 'true',
    'boost' : 'gbdt',
    'metric' : 'rmse'}
model = LGBMRegressor(**parameters)
    
model.fit(
    X_train,
    Y_train, 
    eval_metric="rmse", 
    eval_set=[(X_valid, Y_valid)], 
    verbose=False,
    early_stopping_rounds = 100)

LGBMRegressor(bagging_fraction=0.7135681370918421, bagging_freq=1, boost='gbdt',
              boost_from_average='true', boosting_type='gbdt',
              class_weight=None, colsample_bytree=1.0,
              feature_fraction=0.49446461478601994, importance_type='split',
              learning_rate=0.015980721586917768, max_bin=107, max_depth=19,
              metric='rmse', min_child_samples=20, min_child_weight=0.001,
              min_data_in_leaf=88, min_split_gain=0.0,
              min_sum_hessian_in_leaf=6, n_estimators=3747, n_jobs=-1,
              num_leaves=2000, num_threads=3, objective=None, random_state=None,
              reg_alpha=0.0, reg_lambda=0.0, silent=True, subsample=1.0,
              subsample_for_bin=200000, subsample_freq=0, ...)

In [8]:
data = pd.read_csv("./data/test.csv")
data['item_cnt_month'] = model.predict(X_test).clip(0, 20)
data[['ID', 'item_cnt_month']].to_csv("./features/feature_lgbm_na_tuned.csv", index = False)

In [9]:
pred = model.predict(pd.concat((X_train, X_valid))).clip(0, 20)
data = pd.concat((X_train, X_valid))
data['item_cnt_month'] = pred
data.to_csv("./features_train/feature_lgbm_na_tuned.csv")